In [1]:
from numpy import *
from os import listdir
import operator
import matplotlib
import matplotlib.pyplot as plt

In [2]:
#创建样本数据集和标签向量
def createDataSet():
    group = array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]])
    labels = ['A','A','B','B']
    return group, labels

In [4]:
#inX:输入向量
#dataSet:训练样本集
#labels:标签向量
#k:选择最近邻居的数目
def classify0(inX,dataSet,labels,k):
    #获取样本集的行数
    dataSetSize=dataSet.shape[0]
    #计算距离
    diffMat=tile(inX,(dataSetSize,1))-dataSet
    sqDiffMat=diffMat**2
    sqDistances=sqDiffMat.sum(axis=1)
    distance=sqDistances**0.5
    #对所有的距离进行排序
    sortedDistIndicies=distance.argsort()
    classCount={}
    #选择距离最小的k个点并确定其所在的分类
    for i in range(k):
        voteIlabel=labels[sortedDistIndicies[i]]
        classCount[voteIlabel]=classCount.get(voteIlabel,0)+1
    #排序
    sortedClassCount=sorted(classCount.items(),
    key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

In [8]:
#将文本记录到转换Numpy的解析程序
#filename:输入文本文件名
def file2matrix(filename):
    fr=open(filename)
    arrayOLines=fr.readlines()
    numberOfLines=len(arrayOLines)
    returnMat=zeros((numberOfLines,3))
    classLabelVector=[]
    index=0
    #遍历数组中的数据
    for line in arrayOLines:
        #去除前后空格
        line=line.strip()
        #将每行数据使用\t拆分成列表
        listFromLine=line.split('\t')
        #将列表中的数据复制到returnMat中
        returnMat[index,:]=listFromLine[0:3]
        #将列表中的最后一个数据添加到标签向量中
        classLabelVector.append(int(listFromLine[-1]))
        index+=1
    return returnMat,classLabelVector

In [9]:
#归一化特征值(newValue=(oldValue-min)/(max-min))
def autoNorm(dataSet):
    #最小值
    minVals=dataSet.min(0)
    #最大值
    maxVals=dataSet.max(0)
    #最小值与最大值差（范围）
    ranges=maxVals-minVals
    #与dataSet形状相同的数组数组，用零填充
    normDataSet=zeros(shape(dataSet))
    #数组行数
    m=dataSet.shape[0]
    #将最小值的数据复制为与当前特征值数组长度相同的数组，并用当前特征值数组减去该数组
    normDataSet=dataSet-tile(minVals,(m,1))
    #将范围的数据复制为与当前特征值数组长度相同的数组，并用当前特征值数组除以该数组
    normDataSet=normDataSet/tile(ranges,(m,1))
    return normDataSet,ranges,minVals

In [10]:
#分类器针对约会网站的测试
def datingClassTest():
    #使用百分之十的数据进行分类器测试
    hoRatio=0.10
    #从文件中读取数据
    datingDataMat,datingLabels=file2matrix("datingTestSet2.txt")
    #将数据归一化
    normMat,ranges,minVals=autoNorm(datingDataMat)
    #数组总长度
    m=normMat.shape[0]
    #用于测试分类器的数据条数
    numTestVecs=int(m*hoRatio)
    #错误次数
    errorCount=0.0
    #遍历用于测试分类器的数据将分类器的返回结果与真实结果进行对比，并计算错误率
    for i in range(numTestVecs):
        classifierResult=classify0(normMat[i,:],normMat[numTestVecs:m,:],datingLabels[numTestVecs:m],3)
        print("the classifier came back with: %d, the real answer is:%d"%(classifierResult,datingLabels[i]))
        if(classifierResult!=datingLabels[i]):errorCount+=1.0
    print("the total error rate is:%f"%(errorCount/float(numTestVecs)))

In [11]:
#约会网站预测函数
def classifyPerson():
    #结果列表
    resultList=['not at all','in small doses','in large doses']
    #玩VIdio游戏的时间百分比
    percentTats=float(input("percentage of time spent playing vidio games?"))
    #每年飞行里程数
    ffMiles=float(input("frequent flier miles earned per year?"))
    #每年消耗冰淇淋（升）？
    iceCream=float(input("liters of ice cream consumed per year?"))
    #从文件中读取数据
    datingDataMat,datingLabels=file2matrix("datingTestSet2.txt")
    #将样本数据归一化
    normat,range,minVals=autoNorm(datingDataMat)
    #构造测试分类器的数组
    inArr=array([ffMiles,percentTats,iceCream])
    #将测试数据归一化并获取计算结果
    classifierResult=classify0((inArr-minVals)/range,normat,datingLabels,3)
    print("You will probably like this person:",resultList[classifierResult-1])

In [14]:
#将图像转化为向量
def img2vector(filename):
    #构造1行1024列的矩阵
    returnVect=zeros((1,1024))
    #打开文件
    fr=open(filename)
    #遍历文件的前32行
    for i in range(32):
        #读取文件中的一行
        lineStr=fr.readline()
        #将当前行的前32个数字添加到矩阵的相应位置
        for j in range(32):
            returnVect[0,32*i+j]=int(lineStr[j])
    return returnVect

In [15]:
#手写数字识别系统测试代码
def handwritingClassTest():
    hwLabels=[]
    #获取样本目录中的内容(所有文件名组成的列表)
    trainingFileList=listdir('trainingDigits')
    #目录中的文件数量
    m=len(trainingFileList)
    #构造m行1024列的矩阵
    trainingMat=zeros((m,1024))
    #遍历所有文件
    for i in range(m):
        #获取文件名和扩展名
        fileNameStr=trainingFileList[i]
        #将文件名去掉扩展名
        fileStr=fileNameStr.split(".")[0]
        #文件名前缀（标签）
        classNumStr=int(fileStr.split('_')[0])
        #将文件名前缀添加到标签列表中
        hwLabels.append(classNumStr)
        #将当前文件中的数据读取到列表中，并将列表添加到矩阵中
        trainingMat[i,:]=img2vector("trainingDigits/%s"%fileNameStr)
    #获取测试目录中的内容
    testFileList=listdir("testDigits")
    #错误次数
    errorCount=0.0
    #测试目录中的文件数量
    mTest=len(testFileList)
    for i in range(mTest):
        fileNameStr=testFileList[i]
        fileStr=fileNameStr.split(".")[0]
        classNumStr=int(fileStr.split("_")[0])
        vectorUnderTest=img2vector('testDigits/%s'%fileNameStr)
        #将测试文件数据读取到列表中，并与样本数据进行分类测试
        classifierResult=classify0(vectorUnderTest,trainingMat,hwLabels,3)
        print("the classify0 came back with: %d,the real answer is: %d"%(classifierResult,classNumStr))
        #如果测试分类程序返回值与实际值不相等，错误次数加1
        if(classifierResult!=classNumStr):errorCount+=1
    print("\nthe total number of errors is:%d"%errorCount)
    print("\nthe total error rate is:%f"%(errorCount/float(mTest)))